<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

## Data Processing

In this workflow within PyCCAPT, we can crope the data, do the voltage and bowl calibration, calculate the 3d reconstruction, and do the ranging.
</div>

In [21]:
# Activate intractive functionality of matplotlib
# Activate intractive functionality of matplotlib
%matplotlib ipympl
# Activate auto reload 
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
# import libraries
import os
import numpy as np
from ipywidgets import fixed
from ipywidgets import interact_manual
from ipywidgets import widgets
from IPython.display import display
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

# Local module and scripts
from pyccapt.calibration.calibration_tools import tools, share_variables
from pyccapt.calibration.calibration_tools import widgets as wd
from pyccapt.calibration.data_tools import data_tools, data_loadcrop, dataset_path_qt
from pyccapt.calibration.mc import mc_tools, tof_tools
from pyccapt.calibration.calibration_tools import mc_plot
from pyccapt.calibration.tutorials.tutorials_helper import helper_voltage_bowl
from pyccapt.calibration.tutorials.tutorials_helper import helper_data_loader
from pyccapt.calibration.tutorials.tutorials_helper import helper_temporal_crop
from pyccapt.calibration.tutorials.tutorials_helper import helper_special_crop
from pyccapt.calibration.tutorials.tutorials_helper import helper_t_0_tune
from pyccapt.calibration.tutorials.tutorials_helper import helper_mc_plot
from pyccapt.calibration.tutorials.tutorials_helper import helper_3d_reconstruction
from pyccapt.calibration.tutorials.tutorials_helper import helper_ion_list

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">
In case of recieving the error about pytable library, please install the pytables library by running the cell below. Befor running the cell, uncomment the cell by removing the # sign at the begining of the cell.
</div>


In [2]:
# check pytables library and install it
# try:
#     import tables
# except:
#     !conda install --yes --prefix {sys.prefix} pytables

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">
    
By clicking on the button below, you can select the dataset file you want to crop. The dataset file can be in various formats, including HDF5, EPOS, POS, ATO, and CSV. The cropped data will be saved in the same directory as the original dataset file in a new directory nammed load_crop. The name of the cropped dataset file will be the same as the original dataset file. The figures will be saved in the same directory as the dataset file.
</div>

In [3]:
button = widgets.Button(
    description='load dataset',
)
@button.on_click
def open_file_on_click(b):
    """
    Event handler for button click event.
    Prompts the user to select a dataset file and stores the selected file path in the global variable dataset_path.
    """
    global dataset_path
    dataset_path = dataset_path_qt.gui_fname().decode('ASCII')
button

Button(description='load dataset', style=ButtonStyle())

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

## ROI Selection and Data Cropping Workflow
    

</div>



<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

    
From the dropdown lists below, you can select the instrument specifications of the dataset. The instrument specifications are the same as the ones used for the calibration process. Data mode is specify the dataset structure. The dataset can be in raw or calibrated mode. The flight path length is the distance between the sample and the detector. The t0 is the time of flight of the ions with the lowest mass-to-charge ratio. The maximum mass-to-charge ratio is the maximum mass-to-charge ratio of tat you want to plot. You can also change it in te related cells. The detector diameter is the diameter of the detector.
</div>


In [4]:
# create object for selection of instrument specifications of the dataset
tdc, pulse_mode, flight_path_length, t0, max_mc, det_diam = wd.dataset_instrument_specification_selection()

# Display lists and comboboxes to selected instrument specifications
display(tdc, pulse_mode, flight_path_length, t0, max_mc)

Dropdown(description='Data mode:', options=('PyCCAPT', 'leap_epos', 'leap_pos', 'ato_v6'), value='PyCCAPT')

Dropdown(description='Pulse mode:', options=('voltage', 'laser'), value='voltage')

FloatText(value=110.0, description='Flight path length:')

FloatText(value=38.0, description='t0:')

FloatText(value=400.0, description='Max mc:')

In [10]:
variables = helper_data_loader.load_data(dataset_path, max_mc, flight_path_length, pulse_mode, tdc)

The maximum possible TOF is: 5010 ns
The data will be saved on the path: C:/Users/mehrp/PycharmProjects/pyccapt/tests/data/data_1642_Aug-30-2023_16-05_Al_test4/data_processing/
The dataset name after saving is: data_1642_Aug-30-2023_16-05_Al_test4
The figures will be saved on the path: C:/Users/mehrp/PycharmProjects/pyccapt/tests/data/data_1642_Aug-30-2023_16-05_Al_test4/data_processing/
{'apt': ['high_voltage', 'hour', 'main_chamber_vacuum', 'minute', 'num_events', 'pulse', 'second', 'temperature', 'time_counter'], 'dld': ['high_voltage', 'pulse', 'start_counter', 't', 'x', 'y'], 'tdc': ['channel', 'high_voltage', 'pulse', 'start_counter', 'time_data'], 'time': []}
The number of data over max_tof: 242
Total number of Ions: 12312754


In [11]:
variables.data

,high_voltage (V),pulse,start_counter,t (ns),x_det (cm),y_det (cm)
0,600.000000,328.0,8202,2537.802979,1.080816,0.006531
1,615.000000,328.0,14741,3686.929443,1.443265,-1.812245
2,624.979980,328.0,2657,3110.466553,-0.688980,-2.249796
3,624.979980,328.0,4568,1171.380737,0.192653,-0.914286
4,634.919983,328.0,4498,2703.307129,0.058776,1.479184
...,...,...,...,...,...,...
12312749,8000.000000,1600.0,11089,3722.090332,2.282449,2.798367
12312750,8000.000000,1600.0,13935,3065.292725,3.725714,-0.675918
12312751,8000.000000,1600.0,2722,2561.627686,3.229388,1.573878
12312752,8000.000000,1600.0,3387,3579.656494,0.414694,2.693877


In [12]:
variables.range_data

,ion,mass,mc,mc_low,mc_up,color,element,complex,isotope,charge
0,unranged,0.0,0.0,0.0,400.0,#000000,unranged,0,0,0


<div style="margin: 0 auto; padding: 20px;">

# Temporal crop

Select the data by drawing a rectangle over the experiment history. Experiment history is a 2D histogram of the time of flight of the ions versus sequence of evaporation. The experiment history is plotted by clicking on the button below te cell.
</div>


In [13]:
helper_temporal_crop.call_plot_crop_experiment(variables)

Output()

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

# Spacial crop

Select the region of maximum concentration of Ions in the below plotted graph to utilize relevant data. To crop you can draw a circle over the filed desorption map. The field desorption map is a 2D histogram of the time of flight of the ions versus the position of the ions on the detector. The field desorption map is plotted by clicking on the button below the cell.
</div>

In [14]:
helper_special_crop.call_plot_crop_fdm(variables)

Output()

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

Calculate pulses since the last event pulse and ions per pulse. The percentage of loss in ROI selection process will be printed.
</div>

In [15]:
pulse_pi, ion_pp = data_loadcrop.calculate_ppi_and_ipp(variables.data)

# add two calculated array to the croped dataset
variables.data['pulse_pi'] = pulse_pi.astype(np.uintc)
variables.data['ion_pp'] = ion_pp.astype(np.uintc)

# exctract needed data from Pandas data frame as an numpy array
variables.dld_high_voltage = variables.data['high_voltage (V)'].to_numpy()
variables.dld_pulse = variables.data['pulse'].to_numpy()
variables.dld_t = variables.data['t (ns)'].to_numpy()
variables.dld_x_det = variables.data['x_det (cm)'].to_numpy()
variables.dld_y_det = variables.data['y_det (cm)'].to_numpy()

# save the cropped data
print('tof Crop Loss {:.2f} %'.format((100 - (len(variables.data) / len(variables.data)) * 100)))
#percentage of double event per pulse
print('percentage of double event per pulse', len(ion_pp[ion_pp != 1]) / float(len(ion_pp)))

tof Crop Loss 0.00 %
percentage of double event per pulse 0.01822775720567224


<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

In the next cell by changing the t<sub>0</sub> value you can correct the position of H<sub>1</sub> or any other known peak. this correction would be helpful for the position of the peaks in the m/c calibration process.
</div>

In [20]:
helper_t_0_tune.call_fine_tune_t_0(variables, flight_path_length, pulse_mode, t0)

Output()

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

Remove the data with m/c greater than max m/c and x, y, t = 0. Also add the needed colums for calibration. The data types of the final cropped dataset is displayed below.

</div>

In [17]:
# add the columns in the dataset for x, y, z, mc, mc calibrated, and t calibrated
helper_data_loader.add_columns(variables, max_mc)

display(variables.data)
display(variables.data.dtypes)

The number of data over max_mc: 687857
The number of data with having t, x, and y equal to zero is: 0


,x (nm),y (nm),z (nm),mc_c (Da),mc (Da),high_voltage (V),pulse,start_counter,t_c (ns),t (ns),x_det (cm),y_det (cm),pulse_pi,ion_pp
0,0.0,0.0,0.0,0.0,29.892966,5028.560059,1005.711975,17740,0.0,614.915710,0.943673,1.730612,0,0
1,0.0,0.0,0.0,0.0,29.438919,5028.560059,1005.711975,18142,0.0,625.381042,1.822041,2.671021,402,2
2,0.0,0.0,0.0,0.0,29.262705,5028.560059,1005.711975,18382,0.0,614.181885,2.426122,0.604082,240,1
3,0.0,0.0,0.0,0.0,29.393202,5028.560059,1005.711975,18721,0.0,616.733093,1.678367,1.998367,339,1
4,0.0,0.0,0.0,0.0,29.917120,5028.560059,1005.711975,19550,0.0,608.242859,-0.653061,0.698776,829,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10146658,0.0,0.0,0.0,0.0,30.519060,6347.270020,1269.453979,7993,0.0,552.309021,-0.839184,1.002449,90,1
10146659,0.0,0.0,0.0,0.0,28.771356,6347.270020,1269.453979,8008,0.0,557.966858,3.382857,-0.773878,15,1
10146660,0.0,0.0,0.0,0.0,29.213997,6347.270020,1269.453979,8031,0.0,550.327087,2.174694,1.214694,23,1
10146661,0.0,0.0,0.0,0.0,29.914848,6347.270020,1269.453979,8309,0.0,550.512207,-0.937143,-1.560816,278,1


x (nm)              float64
y (nm)              float64
z (nm)              float64
mc_c (Da)           float64
mc (Da)             float64
high_voltage (V)    float64
pulse               float64
start_counter        uint32
t_c (ns)            float64
t (ns)              float64
x_det (cm)          float64
y_det (cm)          float64
pulse_pi             uint32
ion_pp               uint32
dtype: object

-------------------------------------------------------
-------------------------------------------------------

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

## Time-of-Flight Calibration Workflow

</div>

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">
    
Below you can plot the ToF histogram of the dataset. You can select the peak range of the data you want to plot by drawing a rectangle over peak with holding left click. After that you should apply the voltage and bowl calibration. These three steps should be repeated until you see no improvement in the peak resolution. You can also save the calibration by clicking on the save button. The saved calibration will be used for the next steps.

</div>

In [18]:
# exctract needed data from Pandas data frame as an numpy array
data_tools.extract_data(variables.data, variables, flight_path_length.value, max_mc.value)
calibration_mode = widgets.Dropdown(
    options=[('time_of_flight', 'tof'), ('mass_to_charge', 'mc')],
    description='calibration mode:')
display(calibration_mode)

The maximum time of flight: 5010


Dropdown(description='calibration mode:', options=(('time_of_flight', 'tof'), ('mass_to_charge', 'mc')), value…

In [19]:
helper_voltage_bowl.call_voltage_bowl_calibration(variables, det_diam, flight_path_length, max_mc, calibration_mode)

Output()

In [22]:
helper_ion_list.call_ion_list(variables, selector='peak', calibration_mode=calibration_mode)

In [25]:
helper_mc_plot.call_mc_plot(variables, selector='None')

Output()

In [26]:
variables.data['mc_c (Da)'] = variables.mc_calib
variables.data['t_c (ns)'] = variables.dld_t_calib
# Remove negative mc
threshold = 0
mc_t = variables.data['mc_c (Da)'].to_numpy()
mc_t_mask = (mc_t <= threshold)
print('The number of ions with negative mc are:', len(mc_t_mask[mc_t_mask==True]))
variables.data.drop(np.where(mc_t_mask)[0], inplace=True)
variables.data.reset_index(inplace=True, drop=True)

The number of ions with negative mc are: 10109


-------------------------------------------------------
-------------------------------------------------------

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

## 3D Reconstruction Workflow

After bowl and voltage calibration we are ready to calculate the 3d reconstruction. In this workflow we calculate the reconstructed x,y,z and then plot the 3d, heatmap, projection plots and mass-to-charge histogram.

</div>

In [27]:
# exctract needed data from Pandas data frame as an numpy array
data_tools.extract_data(variables.data, variables, flight_path_length.value, max_mc.value)

display(variables.data)

The maximum time of flight: 5010


,x (nm),y (nm),z (nm),mc_c (Da),mc (Da),high_voltage (V),pulse,start_counter,t_c (ns),t (ns),x_det (cm),y_det (cm),pulse_pi,ion_pp
0,0.0,0.0,0.0,27.301358,29.892966,5028.560059,1005.711975,17740,573.081321,614.915710,0.943673,1.730612,0,0
1,0.0,0.0,0.0,27.093484,29.438919,5028.560059,1005.711975,18142,571.033323,625.381042,1.822041,2.671021,402,2
2,0.0,0.0,0.0,27.160120,29.262705,5028.560059,1005.711975,18382,571.690682,614.181885,2.426122,0.604082,240,1
3,0.0,0.0,0.0,27.025413,29.393202,5028.560059,1005.711975,18721,570.360972,616.733093,1.678367,1.998367,339,1
4,0.0,0.0,0.0,27.025208,29.917120,5028.560059,1005.711975,19550,570.358944,608.242859,-0.653061,0.698776,829,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10136549,0.0,0.0,0.0,27.448261,30.519060,6347.270020,1269.453979,7993,574.523950,552.309021,-0.839184,1.002449,90,1
10136550,0.0,0.0,0.0,26.978264,28.771356,6347.270020,1269.453979,8008,569.894781,557.966858,3.382857,-0.773878,15,1
10136551,0.0,0.0,0.0,26.954607,29.213997,6347.270020,1269.453979,8031,569.660715,550.327087,2.174694,1.214694,23,1
10136552,0.0,0.0,0.0,27.215129,29.914848,6347.270020,1269.453979,8309,572.232732,550.512207,-0.937143,-1.560816,278,1


<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

You have to select the main element in your sample from the from dropdown below.
</div>

In [28]:
element_selected = wd.density_field_selection()
display(element_selected)

Dropdown(description='Element', options=((1, 'H', 52.7, 0.0), (2, 'He', 28.7, 0.0), (3, 'Li', 46.3, 14.0), (4,…

In [29]:
helper_3d_reconstruction.call_x_y_z_calculation(variables, flight_path_length, element_selected)

Output()

In [ ]:
variables.data['x (nm)'] = variables.x
variables.data['y (nm)'] = variables.y
variables.data['z (nm)'] = variables.z
display(variables.data)

-------------------------------------------------------
-------------------------------------------------------

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

## Ion Selection and Rangging

This tutorial outlines a comprehensive workflow for ion selection and organization. Users can choose ions using peak and element finders, manually add ions, customize ion colors, and create histograms. Histograms can be generated for selected ranges and areas, and figures can be saved. Additionally, users have the option to save both figures and data in CSV and HDF5 formats.

</div>

In [ ]:
# exctract needed data from Pandas data frame as an numpy array
data_tools.extract_data(variables.data, variables, flightPathLength.value, max_mc.value)

display(variables.data)